# Análise Igor Malta

# Importação das Bibliotecas

In [1]:
from datetime import datetime
import statistics
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

# Importação dos Dados

In [2]:
clientes = pd.read_csv(r'C:\Users\igor.malta\Downloads\case_localiza\base_clientes.csv',  delimiter = ";")
contratos = pd.read_csv(r'C:\Users\igor.malta\Downloads\case_localiza\base_contratos.csv',  delimiter = ";")

# Limpeza dos dados

### Ao realizar a importação, são boas praticas realizar a conferencia dos primeiros e dos ultimos registros para avaliar se os dados estão corretos, com a separação de colunas e a encodificação corretas, assim como a estrutura do dataframe e suas informações.

In [3]:
clientes.head()

,cd_cliente,nome_cliente,aquisição_cliente
0,1,Rosa,Organic
1,2,Joaquim,Social Media
2,3,Antônio,Direct
3,4,Pedro,Google Ads - Search
4,5,Josefa,Referral


In [4]:
clientes.tail()

,cd_cliente,nome_cliente,aquisição_cliente
44837,109047,Sandra,Organic
44838,112481,Fabio,Organic
44839,114109,Rodrigo,Direct
44840,121495,Sandra,Organic
44841,123883,Pedro,Organic


In [5]:
contratos.head()

,cidade_retirada,cidade_devolucao,cd_contrato,transporte_locado,data_abertura_contrato,qtd_diarias,data_fechamento_contrato,qtd_hora_extra,valor_contrato,cd_cliente
0,Hogsmeade,Hogsmeade,HGM048798,Portal,26/01/2015 05:58,1,27/01/2015 05:58,0,229,1
1,Springfield,Springfield,SPF030834,Ônibus Escolar,05/02/2015 15:10,1,06/02/2015 15:10,0,198,1
2,Porto Real,Porto Real,PRL017782,Barco,04/05/2015 20:49,1,05/05/2015 20:49,0,436,1
3,Basin City,Basin City,SIN042621,1960 Corvette,07/05/2015 07:34,1,08/05/2015 07:34,0,423,1
4,Godric's Hollow,Godric's Hollow,GDH044954,Testralio,01/08/2015 11:14,2,03/08/2015 12:14,1,676,1


In [6]:
contratos.tail()

,cidade_retirada,cidade_devolucao,cd_contrato,transporte_locado,data_abertura_contrato,qtd_diarias,data_fechamento_contrato,qtd_hora_extra,valor_contrato,cd_cliente
480858,Godric's Hollow,Godric's Hollow,GDH015195,Vassoura,25/11/2020 20:25,4,29/11/2020 20:25,0,1204,109047
480859,Springfield,Springfield,SPF004254,Ônibus Escolar,21/03/2018 16:10,7,28/03/2018 17:10,1,2723,112481
480860,Atlântida,Atlântida,ATL054451,Nadadeiras,11/08/2015 18:22,1,12/08/2015 18:22,0,494,114109
480861,Porto Real,Porto Real,PRL054093,Canoa,14/07/2015 02:33,1,15/07/2015 02:33,0,179,121495
480862,Atlântida,Atlântida,ATL031442,Cavalo-Marinho,22/07/2018 00:09,4,26/07/2018 00:09,0,1664,123883


In [7]:
contratos.shape


(480863, 10)

In [8]:
clientes.shape

(44842, 3)

In [9]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44842 entries, 0 to 44841
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cd_cliente         44842 non-null  int64 
 1   nome_cliente       44842 non-null  object
 2   aquisição_cliente  44842 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [10]:
contratos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480863 entries, 0 to 480862
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   cidade_retirada           480863 non-null  object
 1   cidade_devolucao          480863 non-null  object
 2   cd_contrato               480863 non-null  object
 3   transporte_locado         480863 non-null  object
 4   data_abertura_contrato    480863 non-null  object
 5   qtd_diarias               480863 non-null  int64 
 6   data_fechamento_contrato  480863 non-null  object
 7   qtd_hora_extra            480863 non-null  int64 
 8   valor_contrato            480863 non-null  int64 
 9   cd_cliente                480863 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 36.7+ MB


### As bases não possuem valores nulos e está bem estruturada. O próximo passo sera transformar os campos de data que estão com tipo Object para DateTime para que possamos utilizar na agregação min/max e fazer a diferença.

In [11]:

format_str = '%d/%m/%Y %H:%M'
correct_data = lambda x: datetime.strptime(x, format_str)
contratos['data_abertura_contrato'] = list(map(correct_data, contratos['data_abertura_contrato']))
contratos['data_fechamento_contrato'] = list(map(correct_data, contratos['data_fechamento_contrato']))

# Manipulação dos dados

In [12]:
data_churn = contratos.groupby(['cd_cliente']).agg({'data_abertura_contrato':'min','data_fechamento_contrato':'max','qtd_diarias':'mean','cd_cliente':'count','qtd_hora_extra':'mean'})
data_churn['periodo_inicio_fim_dias'] = data_churn['data_fechamento_contrato'] - data_churn['data_abertura_contrato']
data_churn = data_churn.rename(columns={'data_abertura_contrato': 'min_abertura', 'data_fechamento_contrato': 'max_fechamento', 'qtd_diarias': 'media_diarias', 'cd_cliente': 'qtd_contratos', 'qtd_hora_extra': 'media_he'})
data_churn.head()

,min_abertura,max_fechamento,media_diarias,qtd_contratos,media_he,periodo_inicio_fim_dias
cd_cliente,,,,,,
1,2015-01-26 05:58:00,2020-10-21 21:26:00,1.916667,36,0.138889,2095 days 15:28:00
2,2015-01-31 02:29:00,2020-12-15 05:39:00,2.023256,43,0.093023,2145 days 03:10:00
3,2015-02-12 09:27:00,2020-11-01 18:15:00,2.486486,37,0.000000,2089 days 08:48:00
4,2015-03-06 15:37:00,2020-12-14 18:31:00,2.472222,36,0.194444,2110 days 02:54:00
5,2015-02-24 09:00:00,2020-12-12 08:37:00,2.877551,49,0.224490,2117 days 23:37:00


### Os valores da tabela de contratos foram agregados para identificarmos o comportamento de cliente por cliente. Para definir quem são os novos clientes, utilizamos a quantidade de contratos, sendo quantidade 1, um cliente novo. E para o churn, a lógica foi trazer a mediana da diferença das datas de abertura de contrato que aquele cliente ja teve (x - (x-1)).

In [ ]:
contratos = contratos.sort_values(by=['cd_cliente', 'data_abertura_contrato'])
contratos = contratos.reset_index(drop='index')


data_churn['dif_data'] = None
for cliente in contratos.cd_cliente.unique():
    list_datas = contratos[contratos['cd_cliente']==cliente].data_abertura_contrato.to_list()
    list_dif = []
    for index, value in enumerate(list_datas):
        try:
            list_dif.append(list_datas[index+1]-list_datas[index])
        except:
            pass
    try:
        list_dif.sort()
        mediana = statistics.median(list_dif)
        data_churn.loc[data_churn.index==cliente, 'dif_data'] = mediana
    except:
#caso seja apenas um contrato a mediana dará erro, então sera usado o máximo de data da base para relacionar ao fechamento.
#logo, na proxima etapa quando fiz a comparação da mediana geral
        data_churn.loc[data_churn.index==cliente, 'dif_data'] = data_churn.max_fechamento.max()-data_churn.max_fechamento


In [ ]:
data_churn = data_churn.merge(clientes.aquisição_cliente, left_on=data_churn.index, right_on=clientes['cd_cliente'])
data_churn.drop('key_0', axis = 1, inplace = True)

### Foi realizado o join entre a tabela de clientes e a tabela de contratos agregados para termos a informação de origem do cliente em nossas análises.

In [ ]:
data_churn.head()

### Criei as colunas que identificam as duas premissas/regras que precisamos validar, já com as variáveis dummies

In [ ]:
#cliente com um contrato apenas é o novo cliente.

data_churn["categoria_cliente"] = None
data_churn['cliente_novo']=None
data_churn["cliente_antigo"] = None
data_churn.loc[data_churn["qtd_contratos"]==1,'cliente_novo'] = 1
data_churn.loc[data_churn["qtd_contratos"]>1,'cliente_novo'] = 0
data_churn.loc[data_churn["qtd_contratos"]==1,'cliente_antigo'] = 0
data_churn.loc[data_churn["qtd_contratos"]>1,'cliente_antigo'] = 1
data_churn.loc[data_churn["qtd_contratos"]==1,'categoria_cliente'] = "Novo"
data_churn.loc[data_churn["qtd_contratos"]>1,'categoria_cliente'] = "Antigo"

In [ ]:
q2_linha = data_churn.dif_data.quantile(.50)
#mediana da base inteira para se comparar com a do cliente e verificar se é churn ou nao
data_churn['categoria_churn'] = None
data_churn['churn'] = None

data_churn.loc[(data_churn['dif_data'] >= q2_linha),['churn']] = 1
data_churn.loc[(data_churn['dif_data'] < q2_linha),['churn']] = 0
data_churn.loc[(data_churn['dif_data'] >= q2_linha),['categoria_churn']] = "Sim"
data_churn.loc[(data_churn['dif_data'] < q2_linha),['categoria_churn']] = "Não"

# Plotagem dos Histogramas

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].categoria_cliente
dfyes = data_churn[data_churn.categoria_churn == "Sim"].categoria_cliente

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Categoria do cliente")
plt.legend();

## Está validado que, proporcionalmente, o Churn entre os novos clientes é bem maior do que o Churn de clientes mais frequentes. Seria interessante que fosse criado um modelo de fidelização de novos clientes com promoções e demais técnicas para blindagem.

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].media_he
dfyes = data_churn[data_churn.categoria_churn == "Sim"].media_he

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Média de Hora Extra")
plt.legend();

## Segundo o grafico, a frequencia de Churn aumenta suavemente na primeira fração de hora extra. Poderíamos criar uma tolerância para que a primeira hora extra seja cobrada apenas no caso de se completar duas horas.

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].aquisição_cliente
dfyes = data_churn[data_churn.categoria_churn == "Sim"].aquisição_cliente

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Aquisição Cliente")
plt.legend();

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].qtd_contratos
dfyes = data_churn[data_churn.categoria_churn == "Sim"].qtd_contratos

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por qtd Contratos")
plt.legend();

## Assim como no contexto de novos clientes, os clientes que possuem poucos contratos são os que mais abandonam. Incentivos de locação poderiam ser realizados quando os clientes estiverem próximos de alcançar sua média de intervalo entre um contrato e outro.

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].media_diarias
dfyes = data_churn[data_churn.categoria_churn == "Sim"].media_diarias

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Média de Diárias")
plt.legend();

## Os que alugam por menos tempo tambem são os maiores ofensores do churn. Atenção especial para os contratos menores trazendo mais proximidade pós venda com esses clientes, agregando tambem o incentivo que comentamos sobre os de poucos contratos.

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].max_fechamento
dfyes = data_churn[data_churn.categoria_churn == "Sim"].max_fechamento

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Máxima de Fechamento")
plt.legend();

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].min_abertura
dfyes = data_churn[data_churn.categoria_churn == "Sim"].min_abertura

plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.title("Churn por Mínima de Abertura")
plt.legend();

## Os clientes que começaram a locar a partir de 2016 tem uma proporção de churn bem maior que o não churn. Importante verificar mudança de contexto nesse período.

# Balanceamento do Dataset e Criação do modelo de Previsão

In [ ]:
dfno = data_churn[data_churn.categoria_churn == "Não"].categoria_churn
dfyes = data_churn[data_churn.categoria_churn == "Sim"].categoria_churn

total = dfno.count() + dfyes.count()
percent = round((dfno.count()/total)*100,2)
print(f'Sim: {dfyes.count()}', f'Não: {dfno.count()}', f'Não: {percent}')
plt.hist([dfno, dfyes], color=['blue','red'], label=['Churn = Não','Churn = Sim' ])
plt.legend();

### O Dataset está balanceado!!!

In [ ]:
scaler=MinMaxScaler()
data_churn[['media_diarias','qtd_contratos','media_he']] = scaler.fit_transform(data_churn[['media_diarias','qtd_contratos','media_he']])
data_churn.sample(5)

In [ ]:
for col in ['aquisição_cliente','periodo_inicio_fim_dias']:
    scaler = LabelEncoder()
    data_churn[col] = scaler.fit_transform(data_churn[col])
data_churn.sample(5)

### Transformar os campos numericos pra uma escala única para que o modelo de provisão trate os campos sem tendenciar o resultado pela grandeza da variável. Além disso, para que seja treinado um modelo, preciso transformar minhas variáveis categóricas para o tipo numérico, conforme acima.

In [ ]:
#Separar features e labels
features = data_churn.drop(['churn','min_abertura','max_fechamento', 'dif_data','categoria_cliente', 'categoria_churn'],1)
label = data_churn['churn']

#parte para validação
val = data_churn.head(10)

#novo feature
features = features.iloc[10:]
label = label.iloc[10:]

#Separa entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split (features,label, test_size=.30, random_state=0)

#ajustar o tipo de dado das variáveis
y_train=y_train.astype('int64')

### Separação dos dados para criação do modelo de categorização e classificação de churn.

In [ ]:
modelo = RandomForestClassifier(n_estimators = 400 ,random_state = 0, n_jobs = -1)

modelo.fit(x_train,y_train)

In [ ]:
p=modelo.predict(x_test)

x_test['target'] = y_test
y_test=y_test.astype('int64')
x_test['predict'] = p
p=p.astype('int64')

confusion_matrix(y_test, p)

### Por fim, foi realizado o treinamento do modelo e o teste com a comprovação da matriz de confusão. O modelo obteve o resultado de 6023 verdadeiros positivos para 374 falsos positivos, e 6327 verdadeiros negativos para 726 falsos negativos. A precisão minima do modelo em 90%. Segue o relatório de classificação abaixo.

In [ ]:
print(classification_report(y_test, p))